# Product Sales (Research title here)


This is in partial fulfillment of the course Statistical Modeling and simulation (CSMODEL). 

Submitted by: 
- ANG, Charlene
- CAI, Mark Jayson
- SANTOS, Carlo Luis
S15

Submitted to:

Mr. Thomas James Tiam-Lee 

September 23, 2020

# Sales of Summer Clothes in E-commerce Wish

Source: Jeffrey Mvutu Mabilama, https://www.kaggle.com/jmmvutu/summer-products-and-sales-in-ecommerce-wish 

## Introduction

This dataset contains information about available products in eCommerce site Wish. In the data sets are the products' rating and sales information. 

In this notebook, the researchers will be <>

This notebook aims to explore the (research topic)

This may help in _____

### Data Collection

The dataset contains a compiled result of products when the keyword "Summer" is searched in e-commerce website, Wish. An observation is a summer product sold on the platform described using 43 different variables, they are as follows:"

- **`title`**: Title for localized for european countries. May be the same as title_orig if the seller did not offer a translation.
- **`title_orig`**: Original english title of the product.
- **`price`**: Price you would pay to get the product.
- **`retail_price`**: Reference price for similar articles on the market, or in other stores/places.
- **`currency_buyer`**: Currency of the prices.
- **`units_sold`**: Number of units sold. Lower bound approximation by steps.
- **`uses_ad_boosts`**: Whether the seller paid to boost his product within the platform (highlighting, better placement or whatever).
- **`rating`**: Mean product rating.
- **`rating_count`**: Total number of ratings of the product.
- **`rating_five_count`**: Number of 5-star ratings.
- **`rating_four_count`**: Number of 4-star ratings.
- **`rating_three_count`**: Number of 3-star ratings.
- **`rating_two_count`**: Number of 2-star ratings.
- **`rating_one_count`**: Number of 1-star ratings.
- **`badges_count`**: Number of badges the product or the seller have.
- **`badge_local_product`**: A badge that denotes the product is a local product. Conditions may vary (being produced locally, or something else). Some people may prefer buying local products rather than. 1 means Yes, has the badge.
- **`badge_product_quality`**: Badge awarded when many buyers consistently gave good evaluations 1 means Yes, has the badge.
- **`badge_fast_shipping`**: Badge awarded when this product's order is consistently shipped rapidly.
- **`tags`**: Tags of the product set by the seller.
- **`product_color`**: One of the available size variation for this product.
- **`product_variation_inventory`**: Inventory the seller has. Max allowed quantity is 50.
- **`shipping_option_name`**: The name of the shipping options the seller provides.
- **`shipping_option_price`**: Shipping price.
- **`shipping_is_express`**: Whether the shipping is express or not. 1 for True.
- **`countries_shipped_to`**: Number of countries this product is shipped to. Sellers may choose to limit where they ship a product to.
- **`inventory_total`**: Total inventory for all the product's variations (size/color variations for instance).
- **`has_urgency_banner`**: Whether there was an urgency banner with an urgency.
- **`urgency_text`**: A text banner that appear over some products in the search results.
- **`origin_country`**: Where the product was originally developed.
- **`merchant_title`**: Merchant's displayed name (show in the UI as the seller's shop name).
- **`merchant_name`**: Merchant's canonical name. A name not shown publicly. Used by the website under the hood as a canonical name. Easier to process since all lowercase without white space.
- **`merchant_info_subtitle`**: The subtitle text as shown on a seller's info section to the user. (raw, not preprocessed). The website shows this to the user to give an overview of the seller's stats to the user.
- **`merchant_rating_count`**: Number of ratings of this seller.
- **`merchant_rating`**: Merchant's rating.
- **`merchant_id`**: Merchant unique id.
- **`merchant_has_profile_picture`**: Convenience boolean that says whether there is a `merchant_profile_picture` url.
- **`merchant_profile_picture`**: Custom profile picture of the seller (if the seller has one). Empty otherwise.
- **`product_url`**: URL to the product page. You may need to login to access it.
- **`product_picture`**: Picture of the product.
- **`product_id`**: Product identifier. You can use this key to remove duplicate entries if you're not interested in studying them.
- **`theme`**: the search term used in the search bar of the website to get these search results.
- **`crawl_month`**: Meta: for info only.

# Exploratory Data Analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re 
from functools import reduce

### Loading the Dataset 


In [ ]:
df = pd.read_csv("summer-products-with-rating-and-performance_2020-08.csv")
df.head()

### Cleaning the Data

#### Dataset Information

In [ ]:
df.info()

#### Removing Duplicates

Since the dataset was taken from an online source, we must know if there are duplicated information and drop the duplicates if present.

In [ ]:
df.duplicated().sum()

In [ ]:
old_df = df.copy() #copy of df before dropping
df.drop_duplicates(inplace=True)
new_df = df.copy() #copy of df after dropping

#### Check Null Values

In [ ]:
df.isna().sum().sort_values()

##### Profile Picture

Looking at the dataset, merchant_profile_picture is just a link to the picture of the merchant profile picture which no data can be yielded unless one would like to analyze the pixels of the image, however, for this case study, the proponents chose to drop the column along with the boolean column of merchant_has_profile_picture.

In [ ]:
df = df.drop(["merchant_profile_picture", "merchant_has_profile_picture"], axis = 1)

##### Urgency Banner

The column has_urgency_banner contain two unique values which is either 1 or null. The proponents will turn null values into 0 to indicate its lack of an urgency banner to better represent the data.

In [ ]:
df["has_urgency_banner"] = df["has_urgency_banner"].fillna(0)

##### Product Ratings

Checking the product ratings columns, the proponents saw that some observations have missing value for the ratings and that rating for these observations were indicated as 5 even though the indicated rating_count is 0. For these products, the missing values will be indicated as 0 for the rating number counts and actual rating indicating that there has been no reviews for the particular product.

In [ ]:
rating_cols = ["rating_one_count", "rating_two_count", "rating_three_count", "rating_four_count", "rating_five_count", "rating", "rating_count"]

no_votes = df.loc[df[rating_cols].isna().any(axis=1), rating_cols]
df.loc[no_votes.index, rating_cols] = 0

##### Origin Country

In [ ]:
df["origin_country"].value_counts()

The column largely contains the value CN which will not give us much data to be extracted from since there is not a lot of variance in it. The proponents will be dropping the column since there is no significant information to be extracted from it.

In [ ]:
df = df.drop("origin_country", axis=1)

##### Product Color

The missing values for the product_color is hard to fill up with other values therefore, the proponents will just name the null values as the value "null" to make sure they are still categorizable.

In [ ]:
df["product_color"] = df["product_color"].fillna("null")

##### Product Sizes

In [ ]:
df["product_variation_size_id"].value_counts()

The product variation sizes seem to come from different scaling or measurement therefore, the proponents will categorize them only by these standards: "xs", "s", "m", "l", and "xl". The null values will also be replaced with the value of "m" to make sure they are still part of the categorized products.

In [ ]:
def clean_sizes(s: str) -> str:
    return re.findall(r"M|X?[SsLl](?!\w+)", s)

def convert_us_to_eu(s: str) -> str:
    number = re.findall("\d+", s[0])[0]
    
    eu_to_letter = {
        (0, 36): "XS",
        (36, 40): "S",
        (40, 44): "M",
        (44, 48): "L",
        (48, 52): "XL",
        (52, 60): "XXL"
    }
    return [v for k, v in eu_to_letter.items() if k[0]<int(number)<k[1]][0]
     
original_sizes = df["product_variation_size_id"].dropna().unique()
changed_to_letter = [re.sub(r"EU\s*\d+", convert_us_to_eu, s) for s in original_sizes]
filtered_sizes = [clean_sizes(s) for s in changed_to_letter]
original_sizes = df["product_variation_size_id"].dropna()
changed_to_letter = [re.sub(r"EU\s*\d+", convert_us_to_eu, s) for s in original_sizes]
filtered_sizes = [clean_sizes(s) for s in changed_to_letter]
df.loc[original_sizes.index, "product_size"] = [c[0].lower() if c != [] else np.nan for c in filtered_sizes ]
df["product_size"].fillna("m", inplace=True)
df["product_size"].value_counts()

### Exploratory research questions

**Question**: What is the percentage of products that used ad booster 

In [ ]:
boosted = len(df[df["uses_ad_boosts"]==0])
boosted / len(df) * 100

**Question**: 

**Question** : What is the correlation of the ad booster to the sales of a product?

# Research Questions

**Question**: Is there a correlation between the product rating and product sales?


# Di ko mapagbaliktad help!!!

In [ ]:
by_rating = df.groupby("rating").agg({"units_sold": ["mean"]})
by_rating.hist( bins = 40)
plt.show


# DI KO MASORT NA BY MEAN 

In [ ]:
df.groupby("rating").agg({"units_sold": ["mean"]}).sort_values( "rating", ascending = True)

**Question**: Visualize the relationship between the product rating and product sales

**Question**: Find out which product tags are the most effective in regards to unit sold per product 

In [ ]:
df['tags'].value_counts

**Question**: Is there a significant difference between ad-boosted products compared to not boosted in terms of product unit sold

In [ ]:
df.groupby("uses_ad_boosts").agg({"units_sold": ["mean"]})

There is a significant difference between the ad boosted products compared to products that did not use ad_boost

**Question**: Find the confidence interval of mean rating for popular products (in terms of unit sold)


# Insights and Conclusion

With the given data analysis results, we therefore conclude that there is __________________________________